Use Qdrant client, Qdrant is a vector database to store embeedings as vectors

In [4]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct

import pandas as pd
import openai

Read the sampled dataset with Amazon inventory data

In [5]:
df_items = pd.read_json("../../../data/meta_Electronics_2022_2023_with_category_ratings_100_sample_1000.jsonl", lines=True)


In [6]:
df_items.head()

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author
0,Industrial & Scientific,"RAVODOI USB C Cable, [2Pack/3.3ft+6.6ft] USB T...",4.4,119,[【Fast Charging Cord】These USB C cables provid...,[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'Type-C Charger Cable ', 'url': 'ht...",RAVODOI,"[Electronics, Computers & Accessories, Compute...","{'Brand': 'RAVODOI', 'Connector Type': 'USB Ty...",B09R4Y2HKY,NaN,NaN,NaN
1,All Electronics,"SNESH-2 Pack USB-C Female to USB Male Adapter,...",4.5,352,[🔹(Light & compact) Easy to carry and light we...,[],4.99,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'USB Male & Female Adapter', 'url':...",SNESH,"[Electronics, Computers & Accessories, Compute...",{'Package Dimensions': '3.54 x 2.4 x 0.35 inch...,B09JV5FM2S,NaN,NaN,NaN
2,All Electronics,USB C Docking Station Dual Monitor for MacBook...,3.9,1193,[【18-in-1Docking Station】With USB C Docking St...,[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],ZMUIPNG,"[Electronics, Computers & Accessories, Laptop ...","{'Product Dimensions': '3.94""L x 1.18""W x 3.94...",B09SFN9NRX,NaN,NaN,NaN
3,Camera & Photo,[2023 Upgraded] Telescopes for Adults Astronom...,4.1,219,[🎁【2023 All New Experience】The newly upgraded ...,[],169.99,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'Good picture quality', 'url': 'htt...",HUTACT,"[Electronics, Camera & Photo, Binoculars & Sco...","{'Product Dimensions': '32.5""D x 5.5""W x 9.7""H...",B09TP3SZ7C,NaN,NaN,NaN
4,AMAZON FASHION,"Laptop Bag 15.6 Inch, Laptop Briefcase Messeng...",4.5,222,"[Leather,Mesh, Imported, Multi-pockets and Lar...",[],24.95,[{'thumb': 'https://m.media-amazon.com/images/...,[],KPIQIU,"[Electronics, Computers & Accessories, Laptop ...",{'Product Dimensions': '16 x 2 x 12 inches; 1....,B0B5H7T7XZ,NaN,NaN,NaN


Examine the first item of the head recoreds for the feature description

In [7]:
list(df_items["features"].items())[0]

(0,
 ['【Fast Charging Cord】These USB C cables provide up to a 3A charging current to greatly shorten the charging time, meets QC2.0 /3.0 fast charging protocol,Incredibly charge your phone from 0 to 80% in 50 minute. 480Mbps (40-60M/s) ultra fast data transmission, which leads to a faster data sync.(Note:Cables support fast charging,but require a USB-A QC3.0/QC2.0/AFC charger)',
  '【Universal Compatibility】The USB C Charger Cable is compatible with Samsung Galaxy S20 / S10 / S9 / S8+ / S8 / A02s / A03s,A12 A20 A21 A22 A23 A31 A32 A33 A41 A42 A50 A52 A52s 5G A71 A72 A73,M11 M21 M31 M51,M12 M22 M32 M52,iPad Pro 2018 / 2020,Sony Xperia XZ/X Compact/L1 / XZs / XA1 / X Premium, HTC 10 LG G5 G6,OnePlus 5T / 6T, Lumia 950 / 950XL,Huawei P9 P9 Plus P10 P10 Plus Honor Mate 9 Mate 9 pro Mate 10 pro Mate 10 lite and more',
  '【Premium Workmanship】Unique increased friction design allows you to easily unplug the cable from your charger,combine 250d bulletproof fiber core to build a cable so durable

Check the images as well for the first item

In [8]:
list(df_items["images"].items())[0]

(0,
 [{'thumb': 'https://m.media-amazon.com/images/I/51G07yWoOBL._SX38_SY50_CR,0,0,38,50_.jpg',
   'large': 'https://m.media-amazon.com/images/I/51G07yWoOBL.jpg',
   'variant': 'MAIN',
   'hi_res': 'https://m.media-amazon.com/images/I/611AVJMH+JL._SL1200_.jpg'},
  {'thumb': 'https://m.media-amazon.com/images/I/41c+40oKkKL._SX38_SY50_CR,0,0,38,50_.jpg',
   'large': 'https://m.media-amazon.com/images/I/41c+40oKkKL.jpg',
   'variant': 'PT01',
   'hi_res': 'https://m.media-amazon.com/images/I/61ihhPW7VCL._SL1200_.jpg'},
  {'thumb': 'https://m.media-amazon.com/images/I/51y1YnwiUZL._SX38_SY50_CR,0,0,38,50_.jpg',
   'large': 'https://m.media-amazon.com/images/I/51y1YnwiUZL.jpg',
   'variant': 'PT02',
   'hi_res': 'https://m.media-amazon.com/images/I/61UkcVETKcL._SL1200_.jpg'},
  {'thumb': 'https://m.media-amazon.com/images/I/41Nvr++q39L._SX38_SY50_CR,0,0,38,50_.jpg',
   'large': 'https://m.media-amazon.com/images/I/41Nvr++q39L.jpg',
   'variant': 'PT03',
   'hi_res': 'https://m.media-amazon.c

Function to concatenate - Features and title

In [9]:
def preprocess_description(row):
    return f"{row['title']} {' '.join(row['features'])}"

Function to extract first large image from list of images

In [10]:
def extract_first_large_image(row):
    return row["images"][0].get("large", "")

Add new columns for the df_items to have descrption and image

In [11]:
df_items["description"] = df_items.apply(preprocess_description, axis=1)
df_items["image"] = df_items.apply(extract_first_large_image, axis=1)

In [12]:
df_items.head()

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author,image
0,Industrial & Scientific,"RAVODOI USB C Cable, [2Pack/3.3ft+6.6ft] USB T...",4.4,119,[【Fast Charging Cord】These USB C cables provid...,"RAVODOI USB C Cable, [2Pack/3.3ft+6.6ft] USB T...",NaN,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'Type-C Charger Cable ', 'url': 'ht...",RAVODOI,"[Electronics, Computers & Accessories, Compute...","{'Brand': 'RAVODOI', 'Connector Type': 'USB Ty...",B09R4Y2HKY,NaN,NaN,NaN,https://m.media-amazon.com/images/I/51G07yWoOB...
1,All Electronics,"SNESH-2 Pack USB-C Female to USB Male Adapter,...",4.5,352,[🔹(Light & compact) Easy to carry and light we...,"SNESH-2 Pack USB-C Female to USB Male Adapter,...",4.99,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'USB Male & Female Adapter', 'url':...",SNESH,"[Electronics, Computers & Accessories, Compute...",{'Package Dimensions': '3.54 x 2.4 x 0.35 inch...,B09JV5FM2S,NaN,NaN,NaN,https://m.media-amazon.com/images/I/41bOA5-ogW...
2,All Electronics,USB C Docking Station Dual Monitor for MacBook...,3.9,1193,[【18-in-1Docking Station】With USB C Docking St...,USB C Docking Station Dual Monitor for MacBook...,NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],ZMUIPNG,"[Electronics, Computers & Accessories, Laptop ...","{'Product Dimensions': '3.94""L x 1.18""W x 3.94...",B09SFN9NRX,NaN,NaN,NaN,https://m.media-amazon.com/images/I/416IzmVKiC...
3,Camera & Photo,[2023 Upgraded] Telescopes for Adults Astronom...,4.1,219,[🎁【2023 All New Experience】The newly upgraded ...,[2023 Upgraded] Telescopes for Adults Astronom...,169.99,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'Good picture quality', 'url': 'htt...",HUTACT,"[Electronics, Camera & Photo, Binoculars & Sco...","{'Product Dimensions': '32.5""D x 5.5""W x 9.7""H...",B09TP3SZ7C,NaN,NaN,NaN,https://m.media-amazon.com/images/I/41wO4J3TT0...
4,AMAZON FASHION,"Laptop Bag 15.6 Inch, Laptop Briefcase Messeng...",4.5,222,"[Leather,Mesh, Imported, Multi-pockets and Lar...","Laptop Bag 15.6 Inch, Laptop Briefcase Messeng...",24.95,[{'thumb': 'https://m.media-amazon.com/images/...,[],KPIQIU,"[Electronics, Computers & Accessories, Laptop ...",{'Product Dimensions': '16 x 2 x 12 inches; 1....,B0B5H7T7XZ,NaN,NaN,NaN,https://m.media-amazon.com/images/I/41mwlYqT5p...


In [11]:
list(df_items["description"].items())[0]

(0,
 "RAVODOI USB C Cable, [2Pack/3.3ft+6.6ft] USB Type C Fast Charging Cord - Nylon Braided USB C Charger Cable for Galaxy A20/A50/S10/S9/S8+/S8, iPad Pro 2018, Sony XZ, HTC 10, OnePlus 5T, Huawei P9 etc. 【Fast Charging Cord】These USB C cables provide up to a 3A charging current to greatly shorten the charging time, meets QC2.0 /3.0 fast charging protocol,Incredibly charge your phone from 0 to 80% in 50 minute. 480Mbps (40-60M/s) ultra fast data transmission, which leads to a faster data sync.(Note:Cables support fast charging,but require a USB-A QC3.0/QC2.0/AFC charger) 【Universal Compatibility】The USB C Charger Cable is compatible with Samsung Galaxy S20 / S10 / S9 / S8+ / S8 / A02s / A03s,A12 A20 A21 A22 A23 A31 A32 A33 A41 A42 A50 A52 A52s 5G A71 A72 A73,M11 M21 M31 M51,M12 M22 M32 M52,iPad Pro 2018 / 2020,Sony Xperia XZ/X Compact/L1 / XZs / XA1 / X Premium, HTC 10 LG G5 G6,OnePlus 5T / 6T, Lumia 950 / 950XL,Huawei P9 P9 Plus P10 P10 Plus Honor Mate 9 Mate 9 pro Mate 10 pro Mate 1

In [13]:
list(df_items["image"])[0]

'https://m.media-amazon.com/images/I/51G07yWoOBL.jpg'

Sample 50 items from the dataset

In [14]:
df_sample = df_items.sample(50, random_state=42)

In [15]:

len(df_sample)

50

Take specific columns that we are interestd and covert into records. Each element in the record is a dictionary

In [16]:
data_to_embed = df_sample[["description", "image", "rating_number", "price", "average_rating", "parent_asin"]].to_dict(orient="records")


In [17]:
data_to_embed

[{'description': 'KEEPRO Pencil 2nd Generation for iPad, Magnetic Wireless Charge Tilt Sensitivity Palm Rejection Active Pen for Apple iPad Pro 11" 4/3/2/1, iPad Pro 12.9" 6/5/4/3, iPad Air 4/5, iPad Mini 6 [Compatibility]- ONLY compatible with iPad mini (6th generation), iPad Air (4th and 5th generation), iPad Pro 12.9-inch (3rd, 4th, 5th and 6th generation), iPad Pro 11-inch (1st, 2nd, 3rd and 4th generation), check and confirm your device before place the order (Note: If the pen doesn\'t charge, fully charge your iPad first then try charging the pen again) [Charging and Pairs Magnetically]- Charges wirelessly, attaches and pairs magnetically to the compatible iPad, this pen is a preferable alternative to the Apple Pencil 2nd Generation [Tilt Sensitivity & Pixel Precision]- Pixel-perfect precision and industry-leading low latency with tilt sensitivity making drawing, sketching, coloring, taking notes, and marking up PDFs, as easy and natural as a real pencil [Native Palm Rejection]- 

Define Embedding function (random)

In [18]:
response = openai.embeddings.create(
    input="Random text",
    model="text-embedding-3-small",
)


In [19]:
len(response.data[0].embedding)

1536

In [20]:
response

CreateEmbeddingResponse(data=[Embedding(embedding=[-0.015929508954286575, -0.003945682197809219, 0.008374194614589214, 0.003673366503790021, -0.04285980761051178, -0.06279100477695465, -0.007875914685428143, 0.011989621445536613, -0.013024807907640934, -0.006133865565061569, -0.006751887500286102, 0.011811939999461174, -0.020750079303979874, 0.03782292827963829, 0.028428999707102776, -0.018556101247668266, -0.0320444256067276, -0.006670772098004818, -0.05052327364683151, 0.033620379865169525, -0.015504620037972927, 0.024350056424736977, -0.020209310576319695, 7.604563870700076e-05, 0.019127771258354187, -0.0015411917120218277, 0.01582135632634163, -0.008675480261445045, 0.03640148043632507, -0.03637057915329933, 0.024489110335707664, -0.04298340901732445, 0.022465089336037636, -0.025431593880057335, 0.014029093086719513, 0.009123546071350574, 0.050399672240018845, 8.268454985227436e-05, 0.022511441260576248, 0.0023098562378436327, -0.003134528873488307, -0.047433167695999146, 0.0316118

In [21]:
response.data[0].embedding

[-0.015929508954286575,
 -0.003945682197809219,
 0.008374194614589214,
 0.003673366503790021,
 -0.04285980761051178,
 -0.06279100477695465,
 -0.007875914685428143,
 0.011989621445536613,
 -0.013024807907640934,
 -0.006133865565061569,
 -0.006751887500286102,
 0.011811939999461174,
 -0.020750079303979874,
 0.03782292827963829,
 0.028428999707102776,
 -0.018556101247668266,
 -0.0320444256067276,
 -0.006670772098004818,
 -0.05052327364683151,
 0.033620379865169525,
 -0.015504620037972927,
 0.024350056424736977,
 -0.020209310576319695,
 7.604563870700076e-05,
 0.019127771258354187,
 -0.0015411917120218277,
 0.01582135632634163,
 -0.008675480261445045,
 0.03640148043632507,
 -0.03637057915329933,
 0.024489110335707664,
 -0.04298340901732445,
 0.022465089336037636,
 -0.025431593880057335,
 0.014029093086719513,
 0.009123546071350574,
 0.050399672240018845,
 8.268454985227436e-05,
 0.022511441260576248,
 0.0023098562378436327,
 -0.003134528873488307,
 -0.047433167695999146,
 0.031611811369657

Create an embedding function for a text that returns the actual vector map

In [23]:

def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model,
    )
    return response.data[0].embedding



In [24]:
get_embedding("Venkatadri")

[-5.0169688620371744e-05,
 -0.04838788881897926,
 -0.010924947448074818,
 9.08014117158018e-05,
 -0.022017326205968857,
 -0.012564386241137981,
 -0.03691878542304039,
 0.023245161399245262,
 -0.014238707721233368,
 -0.040239524096250534,
 0.04615546017885208,
 0.011999303475022316,
 0.021333644166588783,
 -0.05488983541727066,
 0.057429224252700806,
 0.025407826527953148,
 -0.030919134616851807,
 -0.0032509739976376295,
 -0.009962212294340134,
 0.053634095937013626,
 0.03083541803061962,
 -0.02078949101269245,
 0.07154933363199234,
 0.003760246792808175,
 -0.008664613589644432,
 0.022380094975233078,
 -0.045876406133174896,
 0.019017500802874565,
 0.08176269382238388,
 -0.06021975725889206,
 0.08271147310733795,
 -0.024640429764986038,
 0.0352165587246418,
 -0.03387710079550743,
 -0.0012217314215376973,
 0.06842393428087234,
 0.009773851372301579,
 -0.0025393874384462833,
 -0.06412650644779205,
 0.04339282959699631,
 0.045150864869356155,
 -0.008462299592792988,
 -0.0032265568152070045

Define Qdrant Database and collection

In [25]:
qdrant_client = QdrantClient(url="http://localhost:6333")

In [27]:
qdrant_client.create_collection(
    collection_name="Amazon-items-collection-00",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

True

In [ ]:
qdrant_client.create_collection(
    collection_name="Amazon-items-collection-00",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)


Embed data

In [28]:
pointstruct = PointStruct(
    id=0,
    vector=get_embedding("Venkatadri Ganesan"),
    payload={
        "text": "Test text",
        "model": "text-embedding-3-small",
    },
)

In [29]:
pointstruct

PointStruct(id=0, vector=[0.014939684420824051, -0.034759752452373505, 0.008722387254238129, -0.03631732240319252, 0.0022746999748051167, -0.038004688918590546, -0.06027792766690254, -0.027750691398978233, -0.023973586037755013, -0.03374733403325081, 0.05220452696084976, -6.991822738200426e-05, 0.015965083613991737, -0.03177441284060478, 0.0416649766266346, 0.015393976122140884, -0.02741321735084057, 0.0007163195987232029, 0.0021984439808875322, 0.030216842889785767, 0.05700703337788582, -0.027491096407175064, 0.06951950490474701, 0.017717350274324417, 0.010039831511676311, 0.02192278765141964, -0.010331875644624233, 0.03473379462957382, 0.0573185458779335, -0.03803064674139023, 0.06453528255224228, -0.004312519915401936, -0.01271365862339735, -0.042988911271095276, -0.01732795685529709, 0.03431844338774681, 0.03094370849430561, 0.014926704578101635, -0.056695520877838135, 0.013343175873160362, 0.012467043474316597, -0.00821617804467678, 0.00833299569785595, 0.06925991177558899, 0.0572

Amazon Data

In [36]:
pointstructs = []
for i, data in enumerate(data_to_embed):
    embedding = get_embedding(data["description"])
    pointstructs.append(
        PointStruct(
            id=i,
            vector=embedding,
            payload=data,
        )
    )


In [31]:
pointstructs

[PointStruct(id=0, vector=[0.010930042713880539, -0.015056160278618336, 0.01137808058410883, -0.011284304782748222, -0.0503886416554451, -0.0004936232580803335, -0.03859378397464752, 0.04171963036060333, 0.002547564683482051, -0.010346551425755024, -0.0291745662689209, 0.014462249353528023, -0.03096671774983406, 0.08677349239587784, 0.01424343977123499, 0.004819013178348541, -0.03596807271242142, 0.009127471596002579, -0.018807174637913704, 0.016889989376068115, 0.01028924435377121, 0.0174943208694458, 0.033800818026065826, 0.03059161640703678, 0.021776730194687843, 0.02110988274216652, -0.03313397243618965, -0.026548855006694794, -0.023027068004012108, 0.05222247168421745, -0.013222330249845982, -0.020338840782642365, -0.01941150613129139, -0.049179982393980026, -0.04113613814115524, -0.019265633076429367, -0.017202574759721756, 0.01579594425857067, -0.018254943192005157, 0.0005453950725495815, 0.024214889854192734, 0.04019838199019432, 0.00795527920126915, 0.0011038144584745169, -0.0

In [32]:
len(pointstructs)

50

Write embedded data to Qdrant collection

In [33]:
qdrant_client.upsert(
    collection_name="Amazon-items-collection-00",
    wait=True,
    points=pointstructs,
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

Define function for data retrieval (Top K is the top K nearest search items)

In [34]:
def retrieve_data(query, k=5):
    query_embedding = get_embedding(query)
    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",
        query=query_embedding,
        limit=k,
    )
    return results

Test retrieval

In [35]:
retrieve_data("What kind of charging cords do you offer?", k=10).points



[ScoredPoint(id=4, version=1, score=0.47964922, payload={'description': 'iPhone Charger Cord Lightning Cables, Original 2022 Upgraded [3Pack 3ft] Apple MFi Certified USB A Charging Cable for iPhone 13 12 11 Mini Pro XR Xs Max X SE 8 7 6 Plus iPad iPod AirPods - Black Original Apple Cable - iPhone charging cable with a reinforced joint design that has passed 30,000+ times bending tests for extra protection and durability. Insulation and precisely layer-welded connectors, which make the cable more durable and sturdier than normal iPhone charger cables but also flexible and tangle-free. Instant Responses - No error message pops up. Real high-speed iPhone charging cable with stable and efficient current output, you can plug it into laptop computers and in-car charging adapter. Super Fast Charge - The iPhone Lightning cable build-in four-core high-quality copper wires and multiple safety protections overcharge/stable current/automatic switching/battery protection to promote maximum signal q